In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
join_data = pd.read_excel('C:\\Users\\heeyo\\Desktop\\KERIS_2종\\교육편제단위_표준분류체계 결합.xlsx')
nce_major = pd.read_excel('C:\\Users\\heeyo\\Desktop\\KERIS_2종\\NCE_major.xlsx')

In [3]:
join_data.columns

Index(['조사년도', '조사차수', '학교명', '학교코드', '대학구분', '본분교', '단과대학코드', '학교구분',
       '(대학)지역', '단과대학명', '학교별학과코드', '학부·과(전공)명', '주야구분코드', '주야간구분', '학부특성코드',
       '학과특성', '학과상태', '교육과정', '대계열분류', '중계열분류', '소계열분류', '대학자체대계열', '수업연한',
       '학위과정', '(학과)소재지', '(학과)소재지(상세)', '이수구분', '학점', '교과목해설'],
      dtype='object')

In [4]:
nce_major.columns


Index(['조사년도', '조사차수', '학교명', '학교코드', '대학구분', '본분교', '단과대학코드', '학교구분',
       '(대학)지역', '단과대학명', '학교별학과코드', '학부·과(전공)명', '주야구분코드', '주야간구분', '학부특성코드',
       '학과특성', '학과상태', '교육과정', '대계열분류', '중계열분류', '소계열분류', '대학자체대계열', '수업연한',
       '학위과정', '(학과)소재지', '(학과)소재지(상세)', '이수구분', '학점', '교과목해설', '표본수 n'],
      dtype='object')

In [ ]:
def build_catalog_with_anchor(nce_major: pd.DataFrame,
                              join_data: pd.DataFrame,
                              outpath: str):
    # 1) NCE 학과 목록
    anchors = nce_major[['학교명','학부·과(전공)명']].drop_duplicates()

    # 2) NCE 학과별 교육과정 + 기존 분류
    anchor_courses = (
        join_data.merge(anchors, on=['학교명','학부·과(전공)명'], how='left')
        [['학교명','학부·과(전공)명','대학자체대계열','교육과정',
          '대계열분류','중계열분류','소계열분류']]
        .dropna(subset=['교육과정']).drop_duplicates()
    ).rename(columns={
        '대계열분류':'nce_대계열분류',
        '중계열분류':'nce_중계열분류',
        '소계열분류':'nce_소계열분류'
    })

    # 3) 같은 교육과정을 가진 참고 케이스 (N.C.E. 아님)
    refs = join_data[
        join_data['교육과정'].isin(anchor_courses['교육과정'])
        & (join_data[['대계열분류','중계열분류','소계열분류']] != 'N.C.E').all(axis=1)
    ][['교육과정','학교명','학부·과(전공)명','대학자체대계열',
       '대계열분류','중계열분류','소계열분류']].rename(columns={
        '학교명':'참고_학교명',
        '학부·과(전공)명':'참고_학부·과(전공)명',
        '대학자체대계열':'참고_대학자체대계열',
        '대계열분류':'참고_대계열분류',
        '중계열분류':'참고_중계열분류',
        '소계열분류':'참고_소계열분류'
    })

    # 4) NCE ↔ 참고 매칭
    final_raw = anchor_courses.merge(refs, on='교육과정', how='left')

    # 5) 빈도 집계
    counts = (
        final_raw.groupby([
            '학교명','학부·과(전공)명','대학자체대계열','교육과정',
            'nce_대계열분류','nce_중계열분류','nce_소계열분류',
            '참고_대계열분류','참고_중계열분류','참고_소계열분류'
        ]).size().reset_index(name='빈도')
    )

    # 6) 저장
    os.makedirs(os.path.dirname(outpath), exist_ok=True)
    with pd.ExcelWriter(outpath, engine='xlsxwriter') as w:
        final_raw.to_excel(w, sheet_name='raw', index=False)
        counts.to_excel(w, sheet_name='counts', index=False)

    return final_raw, counts


# 사용 예시
outpath = r"C:\Users\heeyo\Desktop\KERIS_2종\교육과정_분류_카탈로그.xlsx"
raw, counts = build_catalog_with_anchor(nce_major, join_data, outpath)
print("저장 완료:", outpath)


저장 완료: C:\Users\heeyo\Desktop\KERIS_2종\교육과정_분류_카탈로그.xlsx
